___

# [ Machine Learning in Geosciences ] 
Department of Applied Geoinformatics and Carthography, Charles University

Lukas Brodsky lukas.brodsky@natur.cuni.cz


Thi notebook demonstrates rasterization and vectorization (polygomnaztion) processe with Python GDAL/OGR

* 1. Rasterization of polygons ; 

* 2. Vectorization of classification results; 


### Rasterization

In [ ]:
import os
import sys
import matplotlib.pyplot as plt 
%matplotlib inline 
from osgeo import gdal
from osgeo import ogr
from osgeo import osr

#### Data

In [ ]:
# Set your own PATH!!! 
PATH = './data/'

if os.path.isdir(PATH): 
    print('ok')

In [ ]:
raster_fn = os.path.join(PATH, 'landsat.tif')
vector_fn = os.path.join(PATH, 'polygons.shp')

In [ ]:
os.path.isfile(raster_fn)

In [ ]:
os.path.isfile(vector_fn)

In [ ]:
vector_fn

#### Reading vector

In [ ]:
driver = ogr.GetDriverByName('ESRI Shapefile')
datasource = driver.Open(vector_fn, 0)
if datasource is None:
  print('Could not open file')
  sys.exit(1)

In [ ]:
layer = datasource.GetLayer()

#### Reading raster and meatadata 

In [ ]:
ds = gdal.Open(raster_fn)
geo_transform = ds.GetGeoTransform()

x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * ds.RasterXSize
y_min = y_max + geo_transform[5] * ds.RasterYSize
x_res = ds.RasterXSize
y_res = ds.RasterYSize

#### Rasterization

In [ ]:
pixel_width = geo_transform[1]
output_fn = os.path.join(PATH, 'rasterized_poygons.tif')

In [ ]:
# raster target data source to be created
target_ds = gdal.GetDriverByName('GTiff').Create(output_fn, x_res, y_res, 1, gdal.GDT_Byte)
# setting geotransform metadata
target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))

In [ ]:
# Does the file already exists?

In [ ]:
# addd band 
band = target_ds.GetRasterBand(1)
NoData_value = -99
band.SetNoDataValue(NoData_value)
band.FlushCache()

In [ ]:
# What is the content now? 

In [ ]:
# The important step!
gdal.RasterizeLayer(target_ds, [1], layer, options=["ATTRIBUTE=label"])

In [ ]:
# Do not forget!
target_ds = None

In [ ]:
# Check in QGIS! 

### Raster vectorization

In [ ]:
ndsi_fn = os.path.join(PATH, 'landsat_ndsi.tif')
os.path.isfile(ndsi_fn)

In [ ]:
ndsi_ds = gdal.Open(ndsi_fn)
ndsi_band = ndsi_ds.GetRasterBand(1)
ndsi = ndsi_band.ReadAsArray()

In [ ]:
type(ndsi)

In [ ]:
plt.imshow(ndsi, interpolation='none', cmap='RdBu', vmin=-0.3, vmax=0.05)
plt.colorbar()

#### Raster classification map

In [ ]:
glacier = ndsi > -0.1

In [ ]:
plt.imshow(glacier, cmap='RdBu')


In [ ]:
geo_transform = ndsi_ds.GetGeoTransform()
# print(geo_transform)


In [ ]:
projection = ndsi_ds.GetProjectionRef()
# print(projection)


In [ ]:
# save the new image
driver = gdal.GetDriverByName('Gtiff')
filename = os.path.join(PATH, 'glacier_classification.tif')
outDataset = driver.Create(filename, ndsi_ds.RasterYSize, ndsi_ds.RasterXSize, 1, gdal.GDT_Byte)

In [ ]:
outBand = outDataset.GetRasterBand(1).WriteArray(glacier * 1)
outDataset.SetProjection(projection)
outDataset.SetGeoTransform(geo_transform)
outDataset = None

In [ ]:
cls_ds = gdal.Open(filename)
cls_band = cls_ds.GetRasterBand(1)


In [ ]:
# plt.imshow(temp_band.ReadAsArray())

#### Prepare verctor

In [ ]:
driver = ogr.GetDriverByName('ESRI Shapefile')
vector_path = os.path.join(PATH, 'glacier_classification.shp')
out_data = driver.CreateDataSource(vector_path)
layer_name = os.path.basename(vector_path).split('.')[0]
srs = osr.SpatialReference()
srs.ImportFromWkt(ndsi_ds.GetProjectionRef())
out_layer = out_data.CreateLayer(layer_name, srs)

In [ ]:
# create attribute field(s)
fld = ogr.FieldDefn("class", ogr.OFTInteger)
out_layer.CreateField(fld)
dst_field = out_layer.GetLayerDefn().GetFieldIndex("class")


#### Polygonize

In [ ]:
# http://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#polygonize-a-raster-band
gdal.Polygonize(cls_band, cls_band, out_layer, dst_field, ["8CONNECTED=8"], callback=None)
# not yeat saved to file! 

# closing 
out_data = None

cls_ds = None
ndsi_ds = None
